In [4]:
import pandas as pd
import mysql.connector
from mysql.connector.errors import Error
import numpy as np

In [69]:
dados = pd.read_csv('dados/dados_pontos_selecionados.csv').drop_duplicates(subset='old_circuit_id', keep='first').dropna(subset=['new_circuit_id'])

In [65]:
dados['use_cat'].value_counts()

use_cat
Offline    1484
Online      144
Atenção      58
Name: count, dtype: int64

In [5]:
provedores = ['Viasat', 'Hughesnet', 'SES']
probabilities = [0.5, 0.25, 0.25]

# Assign providers randomly based on the probabilities
dados['provedor'] = np.random.choice(provedores, size=len(dados), p=probabilities)


In [8]:
dados['provedor'].value_counts(normalize=True)

provedor
Viasat       0.502372
Hughesnet    0.257414
SES          0.240214
Name: proportion, dtype: float64

In [22]:
dados['last_use'].isna().sum()#.value_counts()

np.int64(702)

In [67]:
target_date = pd.to_datetime('2024-10-31')
random_days = np.random.exponential(2, size=len(dados)).astype(int)
random_days = np.clip(random_days, a_min=0, a_max=30)

vetor = (target_date - pd.to_timedelta(random_days, unit='d'))
dados['last_use'] = dados['last_use'].fillna(pd.Series(vetor.date))

In [72]:
dados['days_since_use'] = (pd.to_datetime('2024-10-31') - pd.to_datetime(dados['last_use'])).dt.days
dados['use_cat'] = dados['days_since_use'].apply(lambda x: 'Online' if x <= 3 else 'Atenção' if x <= 5 else 'Offline')

In [ ]:
dados

,old_circuit_id,new_circuit_id,tipo,status,final_status,modem,plan,mac_address,city,latitude,...,created_at,terminated_at,new_tipo,GBytes_in,GBytes_out,GBytes_total,last_use,days_since_use,use_cat,provedor
0,PASI000016,PASI000016,ubs,Will Not Migrate,Terminated,SB2MODEM,0001-GESAC 10Mbps,NOT_AVAILABLE,ANANINDEUA,-1.405300,...,2019-01-28,2024-01-28,UBS,NaN,NaN,NaN,2024-10-25,6.0,Offline,SES
1,PASI000022,PASI000022,ubs,Will Not Migrate,Terminated,SB2MODEM,0001-GESAC 10Mbps,NOT_AVAILABLE,ABAETETUBA,-1.811900,...,2019-02-05,2024-02-05,UBS,NaN,NaN,NaN,2024-10-13,18.0,Offline,Viasat
2,PASI000026,PASI000026,ubs,Other Providers,Terminated,SB2MODEM,0001-GESAC 10Mbps,NOT_AVAILABLE,BELEM,-1.432222,...,2019-02-08,2024-02-08,UBS,NaN,NaN,NaN,2024-10-25,6.0,Offline,SES
3,PASI000018,PASI000018,ubs,Will Not Migrate,Terminated,SB2MODEM,0001-GESAC 10Mbps,NOT_AVAILABLE,BELEM,-1.365800,...,2019-02-12,2024-02-12,UBS,NaN,NaN,NaN,2024-10-13,18.0,Offline,Viasat
4,PASI000024,PASI000024,ubs,Other Providers,Terminated,SB2MODEM,0001-GESAC 10Mbps,NOT_AVAILABLE,BELEM,-1.303300,...,2019-02-06,2024-02-06,UBS,NaN,NaN,NaN,2024-10-08,23.0,Offline,Viasat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,MASI001835,MASF001236,escola,Migração,Ativo,SB2PLUSMODEM,0042-GESAC2 400 WIFI (SB2+),00A0BC9408D8,NaN,-3.034252,...,2022-04-28,2027-04-28,Escola,150.150631,9.386849,159.537481,2024-10-09,22.0,Offline,SES
1682,MASI001839,MASI001839,escola,Will Not Migrate,Bloqueados,SB2MODEM,0001-GESAC 10Mbps,00A0BC45BB8E,Rosário,-2.934450,...,2022-05-06,2027-05-06,Escola,NaN,NaN,NaN,NaN,NaN,Offline,Viasat
1683,MASI001842,MASF001237,escola,Migração,Ativo,SB2PLUSMODEM,0040-GESAC2 300 (SB2+),00A0BC6BC854,NaN,-2.879630,...,2022-04-27,2027-04-27,Escola,65.828218,4.883339,70.711557,2024-10-27,4.0,Atenção,Viasat
1684,MASI001850,MASF001239,escola,Migração,Ativo,SB2PLUSMODEM,0039-GESAC2 200,00A0BCBE3FE8,NaN,-2.921089,...,2022-04-27,2027-04-27,Escola,107.059018,12.726228,119.785247,2024-10-26,5.0,Atenção,Viasat


In [58]:
dados.to_csv('dados/dados_pontos_selecionados.csv', index=False)

In [ ]:
vdp = pd.read_csv('C:\\Users\\kmenezes\\OneDrive - Viasat, Inc\\Recorrente\\acesso_vdp\\dados\\dados_vdp_nov_21.csv')[['sai', 'idu_device_model', 'root_product_name', 'location', 'mac_address']]
vdp = pd.concat([vdp.drop('location',axis=1), vdp['location'].apply(lambda x: eval(str(x))).apply(pd.Series)], axis=1)
vdp['region'] = vdp['region'].fillna(vdp['sai'].apply(lambda x: x[:2]))
vdp.drop(columns=['country', 'country_code', 'postal_code'], inplace=True)
vdp.columns = ['new_circuit_id', 'modem', 'plan', 'mac_address', 'city', 'latitude', 'longitude', 'state']
vdp['mac_address'] = vdp['mac_address'].str.replace(':','').str.upper()

In [3]:
host = "45.224.130.174"
port = "9773"
user = "karl_menezes"
password = "vK2Tr09Psz"
database = "crm-producao"

connection = mysql.connector.connect(
    host= host,     
    database= database,  
    user= user,   
    password= password,
    port = port  
)

In [12]:
lista = tuple(dados['new_circuit_id'])

sql_query = f''' SELECT
    accounts.circuit_id,
    accounts.email,
    accounts.phone_number_1,
    accounts.phone_number_2,
    installation_validation_data_tipp.accompanying_name,
    installation_validation_data_tipp.email_accompanying,
    installation_validation_data_tipp.tel1_accompanying,
    installation_validation_data_tipp.tel2_accompanying
FROM 
    accounts
LEFT JOIN
    fsm_orders
    ON accounts.id = fsm_orders.account_id
LEFT JOIN
    installation_validation
    ON fsm_orders.id = installation_validation.fsm_order_id
LEFT JOIN
    installation_validation_data_tipp
    ON installation_validation.id = installation_validation_data_tipp.installation_validation_id
WHERE
    accounts.circuit_id IN {lista}
'''

df = pd.read_sql_query(sql_query, connection).drop_duplicates(subset='circuit_id', keep='last')


C:\Users\kmenezes\AppData\Local\Temp\ipykernel_27572\3884493582.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, connection).drop_duplicates(subset='circuit_id', keep='last')


DatabaseError: Execution failed on sql ' SELECT
    accounts.circuit_id,
    accounts.email,
    accounts.phone_number_1,
    accounts.phone_number_2,
    installation_validation_data_tipp.accompanying_name,
    installation_validation_data_tipp.email_accompanying,
    installation_validation_data_tipp.tel1_accompanying,
    installation_validation_data_tipp.tel2_accompanying
FROM 
    accounts
LEFT JOIN
    fsm_orders
    ON accounts.id = fsm_orders.account_id
LEFT JOIN
    installation_validation
    ON fsm_orders.id = installation_validation.fsm_order_id
LEFT JOIN
    installation_validation_data_tipp
    ON installation_validation.id = installation_validation_data_tipp.installation_validation_id
WHERE
    accounts.circuit_id IN ('PASI000016', 'PASI000022', 'PASI000026', 'PASI000018', 'PASI000024', 'ALSF000440', 'PASI001155', 'PASI000127', 'PASI000106', 'PASI000135', 'PASI000136', 'PASI000137', 'PASI000494', 'PASI000495', 'PASI000050', 'PASI000064', 'PASI000088', 'PASI000103', 'PASF

In [11]:
df

,circuit_id,email,phone_number_1,phone_number_2,status
14,RRSI000007,escolafernaodias@gmail.com,9535923024,None,None
21,RSSI000013,pleasechangethisemail@wildblue.net,None,None,None
22,RRSI000173,pleasechangethisemail@wildblue.net,None,None,None
23,RRSI000177,pleasechangethisemail@wildblue.net,None,None,None
24,RRSI000079,pleasechangethisemail@wildblue.net,None,None,None
...,...,...,...,...,...
3066,BASF000311,None,None,None,None
3067,BASF000337,None,None,None,None
3068,RNSF000099,None,None,None,None
3069,TOSF000015,None,None,None,None


In [8]:
df['email'].value_counts().head(15)

email
contato@contato.com.br                   765
contato@contato.com                      280
CONTATO@CONTATO.COM.BR                    58
pleasechangethisemail@wildblue.net        30
romero.lopes@edu.maues.am.gov.br          26
nadilcibatista@gmail.com                  22
gabinete@edu.ma.gov.br                    17
comercial@asapnet.com.br                  14
DEEDUCACAOSECRETARIA@YAHOO.COM.BR         11
jhony.souza@ciodes.ap.gov.br              10
semedbcam@gmail.com                        9
semedrosarioma@gmail.com                   9
jane.pedroso@mcom.gov.br                   6
contato@gmail.com                          5
wellington.souza@portovelho.ro.gov.br      5
Name: count, dtype: int64

In [ ]:
df['phon']

,circuit_id,email,phone_number_1
8,RRSI000007,escolafernaodias@gmail.com,9535923024
15,RSSI000013,pleasechangethisemail@wildblue.net,None
16,RRSI000173,pleasechangethisemail@wildblue.net,None
17,RRSI000177,pleasechangethisemail@wildblue.net,None
18,RRSI000079,pleasechangethisemail@wildblue.net,None
...,...,...,...
1951,BASF000311,None,None
1952,BASF000337,None,None
1953,RNSF000099,None,None
1954,TOSF000015,None,None


In [ ]:
dados = dados.merge(vdp, on='new_circuit_id', how='left')

In [3]:
dados['new_tipo'] = dados['tipo'].apply(lambda x: 'UBS' if x == 'ubs' else 'Escola' if x == 'escola' else 'Povos Originários' if x in ['indigena', 'quilombo'] else 'Outros')

In [5]:
d1 = usage.groupby(['mac_address']).agg({'incomingBytes':'sum', 'outgoingBytes':'sum', 'year':'last', 'month':'last','day':'last'})
d1['last_use'] = pd.to_datetime(d1[['year', 'month', 'day']])
d1['GBytes_in'] = d1['incomingBytes']/(1024**3)
d1['GBytes_out'] = d1['outgoingBytes']/(1024**3)
d1['GBytes_total'] = d1['GBytes_in'] + d1['GBytes_out']
d1 = d1.reset_index()
d1 = d1[['mac_address', 'GBytes_in', 'GBytes_out', 'GBytes_total', 'last_use']]

In [6]:
dados = dados.merge(d1, on='mac_address', how='left')

In [8]:
dados.to_csv('dados_pontos_selecionados.csv', index=False)

In [ ]:
grace = pd.read_csv('C:\\Users\\kmenezes\\OneDrive - Viasat, Inc\\Recorrente\\novo_grace\\sistema 3.0\\verificacao/adfasf.csv')

In [ ]:
d1 = grace.sort_values('completed_date').drop_duplicates(subset='terminal_id', keep='last')[['terminal_id', 'completed_date']]
d1.columns = ['terminal_id', 'created_at']
d1['terminated_at'] = pd.to_datetime(d1['created_at']) + pd.DateOffset(months=60)

In [ ]:
dados.merge(grace[['old_circuit_id', 'terminal_id']].merge(d1, on='terminal_id', how='left'), on = 'old_circuit_id', how='left').to_csv('dados/dados_pontos_selecionados.csv', index=False)

In [4]:
usage = pd.read_csv('dados/oct_hourly_use.csv')

In [ ]:
a1 = usage.groupby(['date']).agg({'Total':'sum'}).reset_index()
a1['Total'] = a1['Total'] / 1024 ** 4

fig2 = px.bar(a1, x='date', y='TBytes', title = 'Consumo de dados por dia do mês', color_discrete_sequence= [c1])

#fig.add_vline(x=37, line_width=3, line_dash="dash", line_color=c2)


fig2.update_xaxes(showgrid=True,  tickangle = 45)


fig2.update_layout(xaxis_title = 'Semana do Ano', yaxis_title = 'Consumo (TBytes)', legend_title = 'Detalhe')

In [ ]:
usage

In [ ]:
usage[usage['month'] == 10].to_csv('dados/oct_hourly_use.csv', index=False)

In [ ]:
apps = pd.read_csv('dados/apps_use.csv')

In [ ]:
apps[apps['month'] == 10].to_csv('dados/oct_apps_use.csv', index=False)